# Семинар 6. Выбор модели. Кросс-валидация. Отбор признаков

Подключение библиотек. 
$
\newcommand{\R}{\mathbb{R}}
\newcommand{\X}{\mathbb{X}}
\newcommand{\norm}[1]{\lVert #1 \rVert}
\newcommand{\abs}[1]{\left| #1 \right|}
\newcommand{\E}{\mathbb{E}}
\newcommand{\D}{\mathbb{D}}
\renewcommand{\Prob}{\mathbb{P}}
\renewcommand{\le}{\leqslant}
\renewcommand{\ge}{\geqslant}
\newcommand{\eps}{\varepsilon}
\newcommand{\Normal}{\mathcal{N}}
\DeclareMathOperator{\TP}{TP}
\DeclareMathOperator{\FP}{FP}
\DeclareMathOperator{\TN}{TN}
\DeclareMathOperator{\FN}{FN}
\DeclareMathOperator{\Accuracy}{Accuracy}
\DeclareMathOperator{\Precision}{Precision}
\DeclareMathOperator{\Recall}{Recall}
\DeclareMathOperator{\Fscore}{F_1}
\DeclareMathOperator{\MSE}{MSE}
\DeclareMathOperator{\RMSE}{RMSE}
\DeclareMathOperator{\MAE}{MAE}
\DeclareMathOperator{\MAPE}{MAPE}
\DeclareMathOperator{\Rsqured}{R^2}
$

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as wg

from matplotlib.colors import ListedColormap

import warnings
warnings.filterwarnings("ignore", 'This pattern has match groups')

RdGn = ListedColormap(['red', 'green'])
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 4)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


def regression_metrics(true, pred, verbose=False, plot=False):
    mae = mean_absolute_error(true, pred)
    mape = np.abs((true - pred) / true).mean()
    mse = mean_squared_error(true, pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(true, pred)

    if verbose:
        print('MAE: ', mae)
        print('MSE: ', mse)
        print('RMSE:', rmse)
        print('MAPE:', mape)
        print('R2:  ', r2)
        print()
    
    if plot:
        plt.figure(0, (6, 6))
        plt.scatter(true, pred)
        low = min(true.min(), pred.min())
        high = max(true.max(), pred.max())

        plt.plot([low, high], [low, high], color='blue')
        plt.xlabel('Ground Truth')
        plt.ylabel('Prediction')
        plt.show()
        
    return (mae, mse, rmse, mape, r2)

# Задача выбора модели

* __Настройка гиперпараметров:__ для конкретного вида модели выбрать наилучшие гиперпараметры (параметры обучения).
* __Выбор вида модели и алгоритма обучения:__ выбрать один из альтернативных видов модели.
* __Отбор признаков:__ отбросить неинформативные признаки.

# 1. Настройка гиперпараметров

Датасет __mpg__. По характеристикам машины необходимо спрогнозировать расход топлива.

Признаки:
* cylinders -- число цилиндров в двигателе;
* displacement -- рабочий объем, л;
* horsepower -- мощность, л.с.;
* weight -- вес, т;
* acceleration -- ускорение;
* model_year -- год выпуска модели;
* origin -- производитель;
* name -- название.

Целевая переменная:
* mpg -- миль на галлон, переведено в км/л.

In [ ]:
frame = sns.load_dataset('mpg')
frame.dropna(inplace=True)

frame['displacement'] *= 16.3871 / 1000
frame['mpg'] = 100 / (frame.mpg * 1.60934 / 3.78541)
frame['weight'] *= 0.453592 / 1000

frame.head()

Данные к обучению:

In [ ]:
X = frame.drop(columns=['mpg', 'name']).copy()
y = frame.mpg.copy()

## 1.1. Метод отложенной выборки

* Делим датасет на 3 части:
    * __Train__ -- используется для обучения параметров модели.
    * __Valid__ -- используется для оценки обобщающей способности модели и для отбора гиперпараметров.
    * __Test__ -- используется для финальной оценки качества алгоритма.

In [ ]:
from sklearn.model_selection import train_test_split


X_fit, X_test, y_fit, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_fit, y_fit, test_size=0.1, random_state=42)

print('Train size:', X_train.shape[0])
print('Valid size:', X_valid.shape[0])
print('Test size:', X_test.shape[0])

Преобразование признаков и модель:

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

num_features = ['displacement', 'horsepower', 'weight', 'acceleration', 'cylinders', 'model_year']
cat_features = ['origin']

transformer = make_column_transformer((StandardScaler(), num_features), (OneHotEncoder(drop='first'), cat_features))
transformer

In [ ]:
from sklearn.linear_model import Ridge


estimator = Ridge(alpha=1e-3)
estimator

In [ ]:
from sklearn.pipeline import make_pipeline


model = make_pipeline(transformer, estimator)
model.fit(X_train, y_train)

pred_train = model.predict(X_train)
pred_valid = model.predict(X_valid)

Печатаем метрики:

In [ ]:
print('Train:')
regression_metrics(y_train, pred_train, verbose=True)

print('Valid:')
regression_metrics(y_valid, pred_valid, verbose=True, plot=True)

Выбор гиперпараметров:

In [ ]:
# Best MAE
from tqdm.auto import tqdm


def func(alpha):
    model = make_pipeline(transformer, Ridge(alpha=alpha))
    model.fit(X_train, y_train)
    
    score = mean_absolute_error
    
    metrics_train = score(y_train, model.predict(X_train))
    metrics_valid = score(y_valid, model.predict(X_valid))
    return metrics_train, metrics_valid

alpha_list = np.logspace(-3, 2, 128)
metrics_list = [func(alpha) for alpha in tqdm(alpha_list)]
train_list, valid_list = zip(*metrics_list)

best_idx = np.argmin(valid_list)
alpha_best = alpha_list[best_idx]
print('Best alpha:', alpha_best)
print('Idx of best alpha:', best_idx)

plt.plot(np.log10(alpha_list), train_list, color='blue', label='Train')
plt.plot(np.log10(alpha_list), valid_list, color='green', label='Valid')
plt.axvline(np.log10(alpha_best), color='purple', lw=2, label='Best alpha')
plt.legend()
plt.show()

Обучаем итоговую модель:

In [ ]:
estimator = Ridge(alpha=alpha_best)

model = make_pipeline(transformer, estimator)
model.fit(X_fit, y_fit)

In [ ]:
print('Fit:')
regression_metrics(y_fit, model.predict(X_fit), verbose=True)

print('Test:')
regression_metrics(y_test, model.predict(X_test), verbose=True, plot=True)

## 1.2. Кросс-валидация

Цели:
* Выбор модели.
* Оценка качества.

Основные разновидности кросс-валидации:
* __Leave One Out:__ 
    * по очереди откладываем одно наблюдение для оценки;
    * обучаем на всей выборке кроме выбранного наблюдения;
    * считаем метрики на отложенном наблюдении.
* __K-Fold:__
    * делим на $k$ непересекающихся групп (fold-ов);
    * далее аналогично Leave One Out.
* __Repeated K-Fold:__
    * несколько раз с разными разбиениями повторяется K-Fold.
* __Stratified K-Fold:__
    * Разбиения пропорционально категориальному признаку (распределение по категориям в fold-ах будет похожим).
* __Grouped K-Fold:__
    * Если одному объекту принадлежит несколько наблюдений (например, пользователь померил несколько моделей обуви).

In [ ]:
X_fit, X_test, y_fit, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print('Fit size:', X_fit.shape[0])
print('Test size:', X_test.shape[0])

$L_2$-регуляризация с кросс-валидацией:

In [ ]:
from sklearn.linear_model import RidgeCV


estimator = RidgeCV(alphas=np.logspace(-3, 2, 64))
model = make_pipeline(transformer, estimator)
model

Настраиваем гиперпараметры и смотрим на качество модели:

In [ ]:
model.fit(X_fit, y_fit)

print('Best alpha:', model['ridgecv'].alpha_)
print('Coefs:', model['ridgecv'].coef_)

In [ ]:
print('Fit:')
regression_metrics(y_fit, model.predict(X_fit), verbose=True)

print('Test:')
regression_metrics(y_test, model.predict(X_test), verbose=True, plot=True)

# 1.3. GridSearch

Умеет перебирать все возможные гипер-параметры и находить оптимальные для произвольных алгоритмов машинного обучения.

In [ ]:
X_fit, X_test, y_fit, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_fit, y_fit, test_size=0.1, random_state=42)

print('Train size:', X_train.shape[0])
print('Valid size:', X_valid.shape[0])
print('Test size:', X_test.shape[0])

In [ ]:
from sklearn.model_selection import GridSearchCV


grid_search = GridSearchCV(Ridge(), {
    'alpha': np.logspace(-3, 2, 32),
    'solver': ('svd', 'cholesky', 'lsqr')
})

model = make_pipeline(transformer, grid_search)
model

In [ ]:
model.fit(X_fit, y_fit)
model['gridsearchcv'].best_estimator_

In [ ]:
print('Fit:')
regression_metrics(y_fit, model.predict(X_fit), verbose=True)

print('Test:')
regression_metrics(y_test, model.predict(X_test), verbose=True, plot=True)

GridSearch для всего пайплайна:

# 2. Выбор вида модели

__Задача:__ отобрать параметры для метода $k$ ближайших соседей.

__Задача:__ 
* обучить понравившуюся Вам модель и подобрать к ней гиперпараметры;
* выбрать метрику качества и сравнить с KNN.

# 3. Отбор признаков

Sklearn обладает инструментами для отбора признаков ([ссылка на документацию](https://scikit-learn.org/stable/modules/feature_selection.html)).

* Удаление признаков с малой дисперсией.
* Отбор признаков с помощью статистических критериев.
* С помощью $L_1$ регуляризации.
* Рекурсивный отбор признаков.
* Последовательный отбор признаков.

In [ ]:
from sklearn.feature_selection import (
    f_regression,
    SelectKBest,
    SelectFromModel,
    RFE,
    SequentialFeatureSelector
)

In [ ]:
X_fit.head()

Выбор с помощью модели:

In [ ]:
from sklearn.linear_model import LinearRegression


features = np.random.normal(size=(128, 16))
targets = 0.1 + 0.05 * features[:, 0] + 0.02 * features[:, 2] - 0.1 * features[:, 4] + np.random.normal(scale=1)

In [ ]:
selector = SelectFromModel(LinearRegression(), max_features=16)
selector.fit(features, targets)

display(selector.get_support())
selector.transform(features)[:5]

Рекурсивный отбор признаков:

In [ ]:
selector = RFE(LinearRegression(), n_features_to_select=6)
selector.fit(features, targets)

display(selector.get_support())
selector.transform(features)[:5]

In [ ]:
selector = SequentialFeatureSelector(LinearRegression(), n_features_to_select=3)
selector.fit(features, targets)

display(selector.get_support())
selector.transform(features)[:5]

Встраиваем в Pipeline:

In [ ]:
selector = SelectKBest(score_func=f_regression, k=5)
estimator = LinearRegression()

model = make_pipeline(transformer, selector, estimator)
model.fit(X_fit, y_fit)

print('Fit:')
regression_metrics(y_fit, model.predict(X_fit), verbose=True)

print('Test:')
regression_metrics(y_test, model.predict(X_test), verbose=True, plot=True)

In [ ]:
selector = SelectKBest(score_func=f_regression, k=1)
estimator = Ridge()

pipeline = make_pipeline(transformer, selector, estimator)
pipeline.fit(X_fit, y_fit)

model = GridSearchCV(pipeline, {
    'ridge__alpha': np.logspace(-4, 3, 16),
    'ridge__fit_intercept': [False, True],
    'columntransformer__standardscaler__with_mean': [False, True],
    'selectkbest__k': list(range(1, 9))
}, verbose=1, n_jobs=16)
model.fit(X_fit, y_fit)
model.best_estimator_

In [ ]:
print('Fit:')
regression_metrics(y_fit, model.predict(X_fit), verbose=True)

print('Test:')
regression_metrics(y_test, model.predict(X_test), verbose=True)

# 4. Оценка качества модели на кросс-валидации

In [ ]:
estimator = Ridge(alpha=0.01)
model = make_pipeline(transformer, estimator)

In [ ]:
from sklearn.model_selection import KFold


cv = KFold(n_splits=5, shuffle=True, random_state=42)

metrics = []

for fit_index, test_index in cv.split(X, y):
    X_fit = X.iloc[fit_index]
    X_test = X.iloc[test_index]
    
    y_fit = y.iloc[fit_index]
    y_test = y.iloc[test_index]
    
    model.fit(X_fit, y_fit)
    pred_test = model.predict(X_test)
    
    metrics.append(regression_metrics(y_test, pred_test))
    
metrics = pd.DataFrame(metrics, columns=['mae', 'mse', 'rmse', 'mape', 'r2'])
metrics

In [ ]:
metrics.mean()

Из коробки:

In [ ]:
from sklearn.model_selection import cross_val_score


cv_result = cross_val_score(model, X, y, cv=cv, scoring='neg_mean_squared_error')
cv_result

In [ ]:
cv_result.mean()

Несколько метрик:

In [ ]:
from sklearn.model_selection import cross_validate


cv_result = cross_validate(model, X, y, cv=cv, 
                           scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'r2'])
cv_result

In [ ]:
{k: np.mean(v) for k, v in cv_result.items()}